Math 5750/6880: Mathematics of Data Science \
Project 3

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.datasets import load_iris, make_moons, load_breast_cancer, fetch_california_housing
from sklearn.datasets import get_data_home
# from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from scipy import io #will use this to load matlab data

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D

# 1. Fashion-MNIST image classification using sklearn

In [2]:
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train1, y_train1), (X_test1, y_test1) = fashion_mnist.load_data()
X_train1 = X_train1.reshape(len(X_train1), -1)
X_test1  = X_test1.reshape(len(X_test1), -1)

# Scale features
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# your code here

https://docs.pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html

# 3. Fashion-MNIST image classification  using pytorch

In [93]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# scale to [0,1], add channel dimension -> (N, 1, 28, 28)# Adding a channel dimension changes image tensors from (N, 28, 28) to (N, 1, 28, 28), matching CNN input shape requirements.
X_train = (X_train.astype("float32") / 255.0)[:, None, :, :]
X_test  = (X_test.astype("float32")  / 255.0)[:,  None, :, :]

y_train = y_train.astype(np.int64)
y_test  = y_test.astype(np.int64)

# train/val split: last 10k of train as validation
X_tr, X_val = X_train[:50000], X_train[50000:]
y_tr, y_val = y_train[:50000], y_train[50000:]

# wrap in PyTorch TensorDatasets and DataLoaders
train_ds = TensorDataset(torch.from_numpy(X_tr),  torch.from_numpy(y_tr)) # Tensors can be created from NumPy arrays: torch.from_numpy(np_array)
val_ds   = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val)) # DataLoader is an iterable that abstracts this complexity for us in an easy API.
test_ds  = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))# DataLoader 是一个可迭代对象，通过简单的 API 帮助我们自动处理数据加载的复杂过程，方便高效地读取批量数据

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)# A batch is like a small “sub-dataset” sampled from the full dataset that the model processes together in one training step.
# shuffle=True randomizes data order each epoch, preventing the model from learning sequence patterns and improving generalization during training.
val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

In [94]:
import torch
import torch.nn as nn
import torch.optim as optim

# In colab, you should ``change runtime type'' to GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# your code here

Using device: cuda


In [95]:
import tensorflow as tf

# "MLP" for adm and....

https://docs.pytorch.org/docs/stable/nn.html

In [359]:
import torch.nn.functional as F

define models

In [362]:
class MLPModel(nn.Module): # inherent from nn.Modle
  def __init__(self):
    super().__init__() #  calls nn.Module constructor
    self.fc1= nn.Linear(28*28, 256) #same indent as calling the parent constructor, 1st fully connected linear layer
    #  also implicitly defines the input dimension
    self.fc2= nn.Linear(256,128) # By default, nn.Linear uses bias=True
    self.fc3=nn.Linear(128,60)
    self.fc4 = nn.Linear(60, 10) # using simialr structure as before
  def forward(self, x):
      x= x.view(-1, 28*28)  # flatten
      x= torch.relu(self.fc1(x))
      x= torch.tanh(self.fc2(x))
      x= torch.relu(self.fc3(x))
      return torch.softmax(self.fc4(x), dim=1) # foward process finally give predictions


In [363]:
model_MLP1 = MLPModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_MLP1.to(device)

MLPModel(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=60, bias=True)
  (fc4): Linear(in_features=60, out_features=10, bias=True)
)

define optimizater and loss

In [364]:
criterion1 = nn.CrossEntropyLoss()
#optimizer1 = torch.optim.LBFGS(
    #model_MLP1.parameters(),
    #lr=0.01,          # moderate learning rate
    #max_iter=160,    # enough inner steps to converge per batch
    #history_size=20, # keeps past updates for curvature estimation
    #line_search_fn='strong_wolfe'  # improves stability
#)


In [365]:
optimizer_1 = torch.optim.Adam(
    model_MLP1.parameters(),
    lr=0.0005,          # standard learning rate
    betas=(0.9, 0.999), # momentum terms for first and second moments
    eps=1e-7,          # small constant for numerical stability
    weight_decay=0.0006     # optional L2 regularization
)


train

In [366]:
import time

In [367]:
def train(model, device, loader, optimizer, criterion, epoch):
    start = time.time()
    model.train() # set in train mode
    for batch_idx, (data, target) in enumerate(loader): # data like input, target like true output labels
        data, target = data.to(device), target.to(device)

        # Closure required for L-BFGS
        def closure():
            optimizer.zero_grad() #梯度清零
            output = model(data) # forward pass
            loss = criterion(output, target) # sum loss
            loss.backward()   # backward pass
            return loss

        loss = optimizer.step(closure) # update parameters by optimize

        if batch_idx % 50 == 0: # report some eporch process and loss per 50 batch
            print(f"Epoch {epoch} [{batch_idx * len(data)}/{len(loader.dataset)}]"
                  f" Loss: {loss.item():.6f}")
    end = time.time()
    print(f"Epoch {epoch} Time: {end-start:.2f}s")

Evaluate and test

In [368]:
def test(model, device, loader, criterion):
    model.eval() # set in evaluate mode
    test_loss, correct = 0, 0
    with torch.no_grad(): # prohibit gradient calculation
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data) # forward pass
            test_loss += criterion(output, target).item() # sum loss
            pred = output.argmax(dim=1, keepdim=True) # get predictions
            correct += pred.eq(target.view_as(pred)).sum().item() # # find the numeber of correct prediction \
    acc = 100. * correct / len(loader.dataset)
    print(f"Average loss: {test_loss/len(loader.dataset):.6f}, validate accuracy: {acc:.2f}%") # (e.g., 0.87) to a percentage (87%)
    return acc

run train and test

In [369]:
train_loader2 = DataLoader(train_ds, batch_size=len(train_ds), shuffle=True)


In [370]:
epochs = 10  # L-BFGS is slower per step; fewer epochs
for epoch in range(1, epochs + 1):
    train(model_MLP1, device, train_loader, optimizer_1, criterion1, epoch)
    test(model_MLP1, device, val_loader, criterion1)

Epoch 1 [0/50000] Loss: 2.302278
Epoch 1 [6400/50000] Loss: 1.969210
Epoch 1 [12800/50000] Loss: 1.842707
Epoch 1 [19200/50000] Loss: 1.770039
Epoch 1 [25600/50000] Loss: 1.708390
Epoch 1 [32000/50000] Loss: 1.646204
Epoch 1 [38400/50000] Loss: 1.635658
Epoch 1 [44800/50000] Loss: 1.712749
Epoch 1 Time: 1.04s
Average loss: 0.006679, validate accuracy: 79.70%
Epoch 2 [0/50000] Loss: 1.647282
Epoch 2 [6400/50000] Loss: 1.706715
Epoch 2 [12800/50000] Loss: 1.670159
Epoch 2 [19200/50000] Loss: 1.707644
Epoch 2 [25600/50000] Loss: 1.638750
Epoch 2 [32000/50000] Loss: 1.617531
Epoch 2 [38400/50000] Loss: 1.693098
Epoch 2 [44800/50000] Loss: 1.655381
Epoch 2 Time: 1.05s
Average loss: 0.006669, validate accuracy: 79.92%
Epoch 3 [0/50000] Loss: 1.657424
Epoch 3 [6400/50000] Loss: 1.672896
Epoch 3 [12800/50000] Loss: 1.643240
Epoch 3 [19200/50000] Loss: 1.705787
Epoch 3 [25600/50000] Loss: 1.628467
Epoch 3 [32000/50000] Loss: 1.695850
Epoch 3 [38400/50000] Loss: 1.628603
Epoch 3 [44800/50000] Lo

In [372]:
epochs = 5  # L-BFGS is slower per step; fewer epochs
for epoch in range(1, epochs + 1):
    train(model_MLP1, device, train_loader, optimizer_1, criterion1, epoch)
    test(model_MLP1, device, val_loader, criterion1)

Epoch 1 [0/50000] Loss: 1.541251
Epoch 1 [6400/50000] Loss: 1.547117
Epoch 1 [12800/50000] Loss: 1.561486
Epoch 1 [19200/50000] Loss: 1.580660
Epoch 1 [25600/50000] Loss: 1.558943
Epoch 1 [32000/50000] Loss: 1.578335
Epoch 1 [38400/50000] Loss: 1.575950
Epoch 1 [44800/50000] Loss: 1.555690
Epoch 1 Time: 1.04s
Average loss: 0.006383, validate accuracy: 86.79%
Epoch 2 [0/50000] Loss: 1.574288
Epoch 2 [6400/50000] Loss: 1.615824
Epoch 2 [12800/50000] Loss: 1.627423
Epoch 2 [19200/50000] Loss: 1.608472
Epoch 2 [25600/50000] Loss: 1.551310
Epoch 2 [32000/50000] Loss: 1.547161
Epoch 2 [38400/50000] Loss: 1.593661
Epoch 2 [44800/50000] Loss: 1.636502
Epoch 2 Time: 1.06s
Average loss: 0.006376, validate accuracy: 87.03%
Epoch 3 [0/50000] Loss: 1.633572
Epoch 3 [6400/50000] Loss: 1.577116
Epoch 3 [12800/50000] Loss: 1.567292
Epoch 3 [19200/50000] Loss: 1.558524
Epoch 3 [25600/50000] Loss: 1.536494
Epoch 3 [32000/50000] Loss: 1.617350
Epoch 3 [38400/50000] Loss: 1.586485
Epoch 3 [44800/50000] Lo

In [373]:
test(model_MLP1, device, test_loader, criterion1)

Average loss: 0.006384, validate accuracy: 86.86%


86.86

In [ ]:
#torch.save(model_MLP1.state_dict(), "MLPModel_LBFGS.pth")
#print("Model saved to MLPModel_LBFGS.pth")

CNN

In [374]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Input: (N, 1, 28, 28)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # -> (N, 32, 28, 28)
        self.pool1 = nn.MaxPool2d(2, 2)  # -> (N, 32, 14, 14), redcue dim
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # -> (N, 64, 14, 14)
        self.pool2 = nn.MaxPool2d(2, 2)                          # -> (N, 64, 7, 7)
        self.fc1 = nn.Linear(64 * 7 * 7, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create model instance
model_1 = CNNModel()
print(model_1)


CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


In [375]:
import time

In [380]:
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # same as SparseCategoricalCrossentropy(from_logits=True)
optimizer_1 = optim.Adam(model_1.parameters(), lr=0.002)

In [381]:

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_1 = model_1.to(device)  #  move model weights to GPU

# Get one batch and move to same device
start_t_CNN_total_1 = time.time()
for epoch in range(5):
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer_1.zero_grad()
        preds = model_1(X_batch)
        loss = loss_fn(preds, y_batch)
        loss.backward()
        optimizer_1.step()
    print(f"Epoch {epoch+1}/5 completed")
end_t_CNN_total_1 = time.time()
total_time_trained=end_t_CNN_total_1-start_t_CNN_total_1
print('total time trained by Pytorch:',total_time_trained)


Epoch 1/5 completed
Epoch 2/5 completed
Epoch 3/5 completed
Epoch 4/5 completed
Epoch 5/5 completed
total time trained by Pytorch: 5.940268039703369


In [382]:
from sklearn.metrics import f1_score, accuracy_score

In [383]:
# ===== VALIDATION =====
model_1.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model_1(X_batch)
        predicted = preds.argmax(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# Compute metrics
acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='macro')

print(f"Validation Accuracy: {acc:.4f}")
print(f"Validation F1 Score (macro): {f1:.4f}")

Validation Accuracy: 0.9225
Validation F1 Score (macro): 0.9225


Above 0.92 is fine-tuning on a pre-trained model with 0.9 accuracy.

In [388]:
X_test2 = torch.from_numpy(X_test).float()
outputs2 = model_1(X_test2.to(device))
y_pred_CNN1 = outputs2.argmax(dim=1).cpu().numpy()



In [389]:
from sklearn.metrics import confusion_matrix

cm_CNN1 = confusion_matrix(y_test, y_pred_CNN1 )
print(cm_CNN1)


[[893   0  14  16   4   1  66   0   6   0]
 [  2 985   0   7   0   0   5   0   1   0]
 [ 26   1 856   8  44   0  64   0   1   0]
 [ 27   2   3 928  28   0  11   0   1   0]
 [  1   0  36  23 864   0  76   0   0   0]
 [  0   0   0   0   0 976   0  17   0   7]
 [127   2  42  25  49   0 750   0   5   0]
 [  0   0   0   0   0   2   0 985   0  13]
 [  4   1   1   4   1   1   1   1 986   0]
 [  1   0   0   0   0   4   0  38   0 957]]


In [390]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_CNN1))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1000
           1       0.99      0.98      0.99      1000
           2       0.90      0.86      0.88      1000
           3       0.92      0.93      0.92      1000
           4       0.87      0.86      0.87      1000
           5       0.99      0.98      0.98      1000
           6       0.77      0.75      0.76      1000
           7       0.95      0.98      0.97      1000
           8       0.99      0.99      0.99      1000
           9       0.98      0.96      0.97      1000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



In [395]:
TP_vec_Adam = np.diag(cm_CNN1)
print("True Positives per class:", TP_vec_Adam)

True Positives per class: [893 985 856 928 864 976 750 985 986 957]


In [397]:
total_TP_Adam = np.trace(cm_CNN1)
total_TP_Adam

np.int64(9180)